In [653]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.rosetta.core.id import *
from pyrosetta.rosetta.core.scoring.constraints import *
import pyrosetta.rosetta.protocols.simple_moves
import numpy as np
import itertools
import scipy
import scipy.linalg
import matplotlib
import matplotlib.pyplot as plt
init('-corrections::restore_talaris_behavior')

Found rosetta database at: /usr/local/lib/python2.7/dist-packages/pyrosetta-2018.11+release.7c7790f-py2.7-linux-x86_64.egg/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2018.11+release.7c7790f7c7790fbb898f8210b54513b9f9c8a927c3e8d7f 2018-03-14T13:19:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.









In [654]:
pmm = PyMOLMover()
pmm.keep_history(True)


In [655]:
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner')
scorefxn_cst = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner')
scorefxn_cst.set_weight(core.scoring.dihedral_constraint, 1.0)

In [656]:
def numpy_hessian(x, h):
        
    x_grad = np.gradient(x, h)
    hessian = np.empty((x.ndim, x.ndim) + x.shape, dtype = x.dtype)
    
    for k, grad_k in enumerate(x_grad):
        tmp_grad = np.gradient(grad_k, h)
        for l, grad_kl in enumerate(tmp_grad):
            hessian[k, l, :, :] = grad_kl
            
    return hessian

In [490]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [491]:
def within_360(dof):
    
    diff = dof/360.0
    diff = np.round(diff)
    
    return dof - 360.0*diff

In [879]:
def is_repeat_minimum(new_dofs, dof_lists):
    
    #dofs_rounded = np.around(new_dofs, decimals=1)
    
    if len(dof_lists) == 0:
        
        return False 
    
    for dof_list in dof_lists:
        
        #dof_list_rounded = np.around(dof_list, decimals=1)

        dof_diff = within_360(new_dofs) - within_360(dof_list)
        
        if np.all(abs(within_360(dof_diff)) < 60.0): return True
        else: continue
    
    return False

In [868]:
def chi_minima(base, sugar_pucker, orientation):

    if base == 'g' or base == 'a':

        if sugar_pucker == 'north':

            if orientation == 'anti': return [72.4]
            else: return [-110.9, -51.9]

        else:

            if orientation == 'anti': return [119.0]
            else: return [-56.6]
            
    else:

        if sugar_pucker == 'north':

            if orientation == 'anti': return [77.3]
            else: return [-105.7, -47.2]

        else:

            if orientation == 'anti': return [108.2]
            else: return [-65.0]        
        


In [869]:
def initialiize_pose(seq):
    
    n_residues = len(seq)
    dof_dict = {}
    
    pose = pyrosetta.rosetta.protocols.recces.pose_setup_turner(seq, '')
    movemap = MoveMap()
    
    for ii in range(n_residues-1):
           
        dof_dict.update({5*ii: TorsionID(ii+1, BB, 5), 5*ii+1: TorsionID(ii+1, BB, 6), 5*ii+2: TorsionID(ii+2, BB, 1), 
                         5*ii+3: TorsionID(ii+2, BB, 2), 5*ii+4: TorsionID(ii+2, BB, 3)})
                        
        movemap.set(TorsionID(5*ii+1, BB, 5), True)
        movemap.set(TorsionID(5*ii+1, BB, 6), True)
        movemap.set(TorsionID(5*ii+2, BB, 1), True)
        movemap.set(TorsionID(5*ii+2, BB, 2), True)
        movemap.set(TorsionID(5*ii+2, BB, 3), True)
    
    for ii in range(n_residues):
    
        dof_dict.update({5*(n_residues-1) + ii: TorsionID(ii+1, CHI, 1)})
        movemap.set(TorsionID(ii+1, CHI, 1), True)
        
    return pose, movemap, dof_dict    
    

In [870]:
def calculate_hessian(pose, scorefxn, dof_dict):
    '''Take a minimized pose and computes the hessian at the base of the corresponding energy well. Returns the hessian matrix
    which may subsequently be diagonalized'''

    minimum = Pose()
    minimum.assign(pose)

    h = 0.1
    h_rad = h*np.pi/180
    E_0 = scorefxn(pose)

    dofs = []

    for val in dof_dict.itervalues(): dofs.append(pose.torsion(val))
    dofs = np.array(dofs)

    hessian = np.zeros((len(dofs), len(dofs)))

    n_pts = int(1/h + 1)
    ind = int(n_pts/2)

    tor_ranges = np.zeros((n_pts, len(dofs)))
    energy = np.zeros((n_pts, n_pts))

    for ii, dof in enumerate(dofs):

        tor_ranges[:, ii] = dof + np.arange(-0.5, 0.5+h, h)

    for pair in list(itertools.combinations(dof_dict.keys(), 2)):

        foo = pair[0]
        bar = pair[1]

        x = tor_ranges[:, foo]
        y = tor_ranges[:, bar]

        for ii, xx in enumerate(x):

            pose.set_torsion(dof_dict[pair[0]], xx)

            for jj, yy in enumerate(y):

                pose.set_torsion(dof_dict[pair[1]], yy)

                energy[ii, jj] = scorefxn(pose)

                pose.set_torsion(dof_dict[pair[1]], dofs[bar])

            pose.set_torsion(dof_dict[pair[0]], dofs[foo])

        hess = numpy_hessian(energy, h_rad)

        d2E_dx2 =  hess[0,0,:,:]
        d2E_dy2 =  hess[1,1,:,:]
        d2E_dxdy = hess[0,1,:,:]

        if hessian[foo, foo] == 0:

            hessian[foo, foo] = d2E_dx2[ind, ind]

        if hessian[bar, bar] == 0:

            hessian[bar, bar] = d2E_dy2[ind, ind]

        hessian[foo, bar] = d2E_dxdy[ind, ind]
        hessian[bar, foo] = d2E_dxdy[ind, ind]
        
    return hessian


In [871]:
def mode_scan(pose, dofs, dof_dict, scorefxn, mode, kT = 1.0, h = 0.5):
    
    temp = Pose()
    temp.assign(pose)
    
    E_0 = scorefxn(pose)
    E = E_0
    inc = 0
    result = np.array([])
    
    h = np.arange(-60,61,1)

    for hh in h:
        
        dofs_new = dofs + hh*mode
        
        for key, val in enumerate(dofs_new):

            temp.set_torsion(dof_dict[key], val)
            
        result = np.append(result, (scorefxn(temp) - E_0)/kT)
     

    #if np.any(result < 0.0): return 1.0
    return np.trapz(np.exp(-result), x=h*np.pi/180.0)


In [872]:
def mode_scan_OG(pose, dofs, dof_dict, scorefxn, mode, kT = 1.0, h = 0.5):
    
    temp = Pose()
    temp.assign(pose)
    
    E_0 = scorefxn(pose)
    E = E_0
    inc = 0
    result = 0.0
    
    max_inc = int(60/h)

    while inc < max_inc:
        
        inc += 1
        result += 0.5*np.exp(-(E-E_0)/kT)
        dofs_new = dofs + (60.0)*inc/float(max_inc)*mode
        
        for key, val in enumerate(dofs_new):

            temp.set_torsion(dof_dict[key], val)
        
        E = scorefxn(temp)
        if E < E_0: break
        result += 0.5*np.exp(-(E-E_0)/kT)

    if inc < max_inc: return 1.0
    # Reinitialize pose
    temp.assign(pose)
    inc = 0
    E = E_0
    
    while inc < max_inc:
        
        inc += 1
        
        result += 0.5*np.exp(-(E-E_0)/kT)
        dofs_new = dofs + 60.0*inc/float(max_inc)*mode
        
        for key, val in enumerate(dofs_new):
    
            temp.set_torsion(dof_dict[key], val)
        
        E  = scorefxn(temp)
        if E < E_0: break
        result += 0.5*np.exp(-(E-E_0)/kT)
        
    result *= h*np.pi/180.0
    
    if inc < max_inc: return 1.0
    return result


In [873]:
def assay_modes(pose, scorefxn, dof_dict):
    
    '''this code will assay the normal modes, plotting the corresponding boltzmann weights for both the harmonic approximation
    and scanned modes'''


    import scipy.integrate
    
    pose.assign(pose_min)
    print scorefxn(pose)

    hessian = calculate_hessian(pose, scorefxn, dof_dict)

    dofs = []

    for val in dof_dict.itervalues(): dofs.append(pose.torsion(val))
    dofs = np.array(dofs)

    w, v = scipy.linalg.eigh(hessian)



    E_min = scorefxn(pose)
    kT = 1.0
    fig = plt.figure()
    for kk, K in enumerate(w):

        mode = v[:,kk]
        h = np.arange(-60,61,1)
        h_plot = h*np.pi/180

        bar = []


        for hh in h:

            dofs_new = dofs+hh*mode

            for jj, dof in enumerate(dofs_new):

                pose.set_torsion(dof_dict[jj], dof)

            pmm.apply(pose)
            bar.append((scorefxn(pose)-E_min)/kT)


        np.array(bar) 
        pose.assign(minimum)
        plt.subplot(10,5, kk+1)
        plt.figure()
        plt.plot(h, np.exp(-np.array(bar)))
        plt.plot(h, np.exp(-0.5*K*h_plot**2/kT))
        #plt.xticks([])
        #plt.yticks([])
        #plt.xlabel('Displacement along normal coordinate (degrees)')
        #plt.ylabel('exp(-(E-E0)/kT)')


In [874]:
def add_constraints(pose):
    pose.add_constraint(
        core.scoring.constraints.DihedralConstraint(AtomID(11,2), AtomID(10,2), AtomID(23,2), AtomID(17, 2),
        core.scoring.func.CircularSigmoidalFunc(0.5, 6, pose.chi(2))))#CircularHarmonicFunc(pose.chi(2)*np.pi/180., 0.35)))
    pose.add_constraint(
        core.scoring.constraints.DihedralConstraint(AtomID(11,1), AtomID(10,1), AtomID(23,1), AtomID(17, 1),
        core.scoring.func.CircularSigmoidalFunc(0.5, 6, pose.chi(1))))#CircularHarmonicFunc(pose.chi(1)*np.pi/180., 0.35)))
    #pose.add_constraint(
    #    core.scoring.constraints.DihedralConstraint(AtomID(6,1), AtomID(8,1), AtomID(9,1), AtomID(1,2), 
    #    core.scoring.func.CircularHarmonicFunc(pose.epsilon(1)*np.pi/180., 0.35)))
    #pose.add_constraint(
    #    core.scoring.constraints.DihedralConstraint(AtomID(8,1), AtomID(9,1), AtomID(1,2), AtomID(4,2),
    #    core.scoring.func.CircularHarmonicFunc(pose.zeta(1)*np.pi/180., 0.35)))
    #pose.add_constraint(
    #    core.scoring.constraints.DihedralConstraint(AtomID(9,1), AtomID(1,2), AtomID(4,2), AtomID(5,2),
    #    core.scoring.func.CircularHarmonicFunc(pose.alpha(2)*np.pi/180., 0.35)))
    #pose.add_constraint(
    #    core.scoring.constraints.DihedralConstraint(AtomID(1,2), AtomID(4,2), AtomID(5,2), AtomID(6,2),
    #    core.scoring.func.CircularHarmonicFunc(pose.beta(2)*np.pi/180., 0.25)))
    #pose.add_constraint(
    #    core.scoring.constraints.DihedralConstraint(AtomID(4,2), AtomID(5,2), AtomID(6,2), AtomID(8,2),
    #    core.scoring.func.CircularHarmonicFunc(pose.gamma(2)*np.pi/180., 0.25)))

    return pose

In [875]:
def MC_minimization(pose, scorefxn, dof_dict, torsion_list):
    
    import random
    mc = MonteCarlo(pose, scorefxn, 0.5)
    
    for i in range(100):

        dof = random.randint(0, 6)
        new_dof = random.uniform(torsion_list[dof] - 30.0, torsion_list[dof] + 30.0)
        pose.set_torsion(dof_dict[dof], new_dof)
        mc.boltzmann(pose)
    
    return pose
    

In [880]:
seq = 'gc'

# Pucker and orientation of terminal base
pucker = 'north'
orientation = 'anti'

n = len(seq)
Z_harm_list = np.array([])
mu_list = np.array([])
pmm = PyMOLMover()
pmm.keep_history(True)
dof_lists = []
E_list = np.array([])
E_min = 0
pose_min = Pose()
dofs_min = []

In [881]:
# Code for any dinucleotide 
for delta in [81.195, 145.521]:
    
    for alpha in [-64.3, 66.7, 156.6]:
        
        pose, movemap, dof_dict = initialiize_pose(seq)
        
        pose.set_torsion(TorsionID(1, BB, 4), delta)
    
        if delta == 81.195:

            if seq[0] in 'ga': possible_chi = [-110.9, -51.9, 72.4]
            else: possible_chi = [-105.7, -47.2, 77.3]
                    
            possible_eps = [-150.4, 16.8, 76.2]

        #C2'-endo
        else:

            if seq[0] in 'ga': possible_chi = [-56.6, 119.0]
            else: possible_chi =[-65, 108]
                
            possible_eps = [-97.4, 0, 75.2]
            
            pose.set_torsion(TorsionID(1, CHI, 2), -34.825)
            pose.set_torsion(TorsionID(1, CHI, 3), 155.987)
                
        #if pucker == 'north':
        
            #pose.set_torsion(TorsionID(2, BB, 4), 81.195)
            #pose.set_torsion(TorsionID(2, CHI, 2), 35.870)
            #pose.set_torsion(TorsionID(2, CHI, 3), 94.856)
            
        if pucker == 'south':
        
            # These will set the terminal base pair to C2'-endo (south)
            pose.set_torsion(TorsionID(2, BB, 4), 145.521)
            pose.set_torsion(TorsionID(2, CHI, 2), -34.79)
            pose.set_torsion(TorsionID(2, CHI, 3), 156.00)

        if alpha == -64.3: possible_zeta = [-71.8, 167.55]
        elif alpha == 66.7: possible_zeta = [-131.6, 79.0]
        else: possible_zeta = [-69.5, 56.5]
        
        possible_chi_2 = chi_minima(seq[-1], pucker, orientation)
        torsion_lists = itertools.product(possible_eps, possible_zeta, [alpha], [177.3], [175.8, 53.5, -62.9], possible_chi, 
                                          possible_chi_2)
        
        
        
        for torsion_list in list(torsion_lists):
            
            pose.set_torsion(TorsionID(1, BB, 5), torsion_list[0])
            pose.set_torsion(TorsionID(1, BB, 6), torsion_list[1])
            pose.set_torsion(TorsionID(2, BB, 1), torsion_list[2])
            pose.set_torsion(TorsionID(2, BB, 2), torsion_list[3])
            pose.set_torsion(TorsionID(2, BB, 3), torsion_list[4])
            pose.set_torsion(TorsionID(1, CHI, 1), torsion_list[5])
            pose.set_torsion(TorsionID(2, CHI, 1), torsion_list[6])

            pmm.apply(pose)
            
            pose = add_constraints(pose)
            
            pose = MC_minimization(pose, scorefxn, dof_dict, torsion_list)
            
            minmover = rosetta.protocols.minimization_packing.MinMover(
                movemap, scorefxn, 'dfpmin_armijo_nonmonotone', 1.0e-7, True)
            minmover.max_iter(1000000)
            minmover.min_options().use_nblist(True)
            minmover.min_options().nblist_auto_update(True)
            minmover.apply(pose)

            pmm.apply(pose)
            E_min = scorefxn(pose)
            
            minimum = Pose()
            minimum.assign(pose)

            dofs = []

            for val in dof_dict.itervalues(): dofs.append(pose.torsion(val))
            dofs = np.array(dofs)


            if is_repeat_minimum(dofs, dof_lists): continue
        
            dof_lists.append(dofs)
                    
            hessian = calculate_hessian(pose, scorefxn, dof_dict)
            w,v = scipy.linalg.eigh(hessian)

            if not np.all(w > 0.):
                continue

            E_list = np.append(E_list, E_min)
            
            Z = 1.0
            Z_harm = np.sqrt((2*np.pi)**len(w)/scipy.linalg.det(hessian)) 
            
            for kk, K in enumerate(w):

                fuck = mode_scan(pose, dofs, dof_dict, scorefxn, v[:, kk])
                
                if fuck == 1.0: break
                    
                Z *= fuck

            if kk < 6:
                
                dof_lists.pop()
                E_lists = np.delete(E_list, -1)
                E_min = 0
                
                continue
            
            if np.all(E_list >= E_min):
                
                pose_min.assign(minimum)
                dofs_min = dofs
                
            #Z_list = np.append(Z_list, np.exp(-scorefxn(minimum)/0.6163)*Z )
            Z_harm_list = np.append(Z_harm_list, np.exp(-scorefxn(minimum))*Z_harm)
            mu_list = np.append(mu_list, scorefxn(minimum) - np.log(Z))
            print scorefxn(minimum) - np.log(Z), Z, scorefxn(minimum)
            #print -0.6163*np.log(np.sum(np.exp(-mu_list/0.6163))), scorefxn(minimum), np.exp(-(scorefxn(minimum) - np.log(Z))/0.6163)

'''
# South, anti
pose.set_torsion(TorsionID(1, BB, 4), 145.4)
pose.set_torsion(TorsionID(1, BB, 5), -97.4)
pose.set_torsion(TorsionID(1, CHI, 1), -56.6)
pose.set_torsion(TorsionID(1, CHI, 2), -37.2)
pose.set_torsion(TorsionID(1, CHI, 3), 155.5)
pose.set_torsion(TorsionID(2, BB, 1), 66.7)
pose.set_torsion(TorsionID(2, BB, 3), -62.9)
pose.set_torsion(TorsionID(2, BB, 4), 145.4)
pose.set_torsion(TorsionID(2, CHI, 1), -56.6)
pose.set_torsion(TorsionID(2, CHI, 2), -37.2)
pose.set_torsion(TorsionID(2, CHI, 3), 155.5)
'''

9.848868854115644 0.5200484984257436 9.19503564856
4.608574068735531 0.010675649646736098 0.0687842038535
9.975908558392176 0.00073130349178073 2.75522654745
-0.7780401352753685 0.0003919469466235811 -8.62242420285
12.221406278614516 0.08811449229538156 9.79228801729
6.153738234593439 0.0002687989768593236 -2.06780852106
10.438021753693496 2.6527536411515076 11.4136199641
10.339209326498063 0.11889628531622447 8.20970560864
4.360869791377924 0.3087246207597715 3.18556419699
9.530949492400921 2.684351000525767 10.5183884781
9.288849487881507 0.14200802740526056 7.33697779593
3.4621246385519835 0.3096810788297079 2.28991234953
11.956831392521677 2.390872152809242 12.8284896094
11.944213537829619 0.08550684327627212 9.48505466991
5.8904776559378424 0.27520949660026545 4.60025499041
16.625213635098127 0.0580718162567719 13.779138812
9.820251112895551 0.013952209652994943 5.5481337277
16.742046219744832 0.013416017394279146 12.4307402618
9.917870979411573 0.003304963362373529 4.20554108816


'\n# South, anti\npose.set_torsion(TorsionID(1, BB, 4), 145.4)\npose.set_torsion(TorsionID(1, BB, 5), -97.4)\npose.set_torsion(TorsionID(1, CHI, 1), -56.6)\npose.set_torsion(TorsionID(1, CHI, 2), -37.2)\npose.set_torsion(TorsionID(1, CHI, 3), 155.5)\npose.set_torsion(TorsionID(2, BB, 1), 66.7)\npose.set_torsion(TorsionID(2, BB, 3), -62.9)\npose.set_torsion(TorsionID(2, BB, 4), 145.4)\npose.set_torsion(TorsionID(2, CHI, 1), -56.6)\npose.set_torsion(TorsionID(2, CHI, 2), -37.2)\npose.set_torsion(TorsionID(2, CHI, 3), 155.5)\n'

In [882]:
import scipy.special
print 0.6163*(-scipy.special.logsumexp(-mu_list[:]))
print -0.6163*np.log(np.sum(Z_harm_list))
fuck = Z_list*(2*np.pi/3)**7
print -0.6163*np.log(np.sum(fuck))
print -0.6163*np.log(np.sum(Z_list)) 
print np.sum(Z_list)
print np.sum(Z_list*(2*np.pi/3)**7)
print len(Z_list)
print -0.6163*np.log(np.sum(Z_list))
print np.sum(Z_harm_list)
print np.sum(Z_list) 
print Z_harm_list

-0.6233583213728926
-0.5653629766764225
inf
inf
0.0
0.0
0
inf
2.50265027050891
0.0
[2.68542595e-06 8.12835268e-04 3.07485112e-05 2.17630655e+00
 5.81795317e-07 1.56878736e-03 6.08331018e-07 1.54833440e-05
 3.45237343e-03 1.63556166e-06 2.48179736e-05 7.78516788e-03
 1.58282034e-07 3.22993587e-06 8.82274059e-04 1.17682269e-08
 5.45338206e-05 9.48440888e-09 4.36463268e-05 1.86221625e-07
 1.62774767e-08 1.01976309e-05 4.59205217e-09 2.13352306e-05
 9.54311923e-09 4.43212671e-05 1.17173437e-09 5.49699091e-06
 1.18776180e-05 1.92979226e-06 1.11935807e-02 4.63828614e-06
 3.89481254e-05 3.74105828e-02 9.23550562e-07 6.05511680e-06
 2.27515244e-04 1.71066753e-06 1.67378595e-05 1.10016928e-03
 1.27054381e-05 1.38252159e-02 8.13552123e-07 3.11901603e-03
 8.61394635e-09 4.21185360e-05 1.32963878e-08 5.81493777e-05
 3.12674337e-09 1.36715599e-05 1.42503318e-10 7.02985070e-07
 8.58303193e-10 4.29202545e-06 4.77784231e-10 2.28619541e-06
 2.47159824e-08 3.38526693e-08 1.07347005e-04 1.24716725e-06
 8

/home/maa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/home/maa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/home/maa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


In [883]:
#hist = plt.hist(E_list,bins=9000,range=(-100,800))
print -0.6163*np.log(np.sum(np.exp(-mu_list)))

print pucker, orientation

-0.623358321372893
north anti


In [827]:
from scipy.special import logsumexp
from recces.data import SingleHistSimulation
from recces.util import torsion_volume
print min(E_list)
sim = SingleHistSimulation('./../recces_rosetta/results/gg/ST/')
plt.plot(hist[1][500:1500], hist[0][500:1500]*100)#*np.exp(-hist[1][500:1500])/75)
plt.plot(sim._dos_scores[500:1500], sim._dos[500:1500])#*np.exp(-sim._dos_scores[500:1500]))

-6.532844149973429
Inferring sequence from directory name:  gg
applying phase space volume for single strand with length  2


NameError: name 'hist' is not defined

In [ ]:
'''this code will assay the normal modes, plotting the corresponding boltzmann weights for both the harmonic approximation
and scanned modes'''


pose.assign(pose_min)
print pose_min
print scorefxn(pose)

hessian = calculate_hessian(pose, scorefxn, dof_dict)

dofs = []
configuration_integral = 1.0

for val in dof_dict.itervalues(): dofs.append(pose.torsion(val))
dofs = np.array(dofs)
 
w, v = scipy.linalg.eigh(hessian)

import scipy.integrate

E_min = scorefxn(pose)
kT = 1.0
fig = plt.figure()
for kk, K in enumerate(w):
 
    mode = v[:,kk]
    h = np.arange(-60,61,1)
    h_plot = h*np.pi/180.0

    bar = []


    for hh in h:

        dofs_new = dofs+hh*mode

        for jj, dof in enumerate(dofs_new):

            pose.set_torsion(dof_dict[jj], dof)

        pmm.apply(pose)
        bar.append((scorefxn(pose)-E_min)/kT)
        

    np.array(bar) 
    configuration_integral *= np.trapz(np.exp(-np.array(bar)), x=h_plot[:])
    print np.trapz(np.exp(-np.array(bar)), x=h_plot[:])
    pose.assign(pose_min)
    #plt.subplot(10,5, kk+1)
    plt.figure()
    plt.plot(h, np.exp(-np.array(bar)))
    plt.plot(h, np.exp(-0.5*K*h_plot**2/kT))
    #plt.xticks([])
    #plt.yticks([])
    #plt.xlabel('Displacement along normal coordinate (degrees)')
    #plt.ylabel('exp(-(E-E0)/kT)')


In [325]:
print scorefxn(pose_min) - 0.6163*np.log(configuration_integral)
print configuration_integral
print -0.6163*np.log(np.exp(-scorefxn(pose_min)/0.6163)*configuration_integral) 
print -0.6163*scipy.special.logsumexp(-mu_list)
print -0.6163*np.log(np.exp(-scorefxn(pose_min)/0.6163)*configuration_integral)
print scorefxn(pose_min) - np.log(configuration_integral)
print (np.exp(-(scorefxn(pose_min)-np.log(configuration_integral))/0.6163))
print

-4.42531654399566
0.00021986977826110677
-4.42531654399566
-2.7323508653852295
-4.42531654399566
-1.1936128466449976
6.936102206922783



In [442]:
pose = pose_from_file('./../gg_fuck.pdb')
huh = pose
minmover.apply(pose)
pmm.apply(pose)
hessian = calculate_hessian(pose_min, scorefxn, dof_dict)
print scorefxn(huh)
dofs = []
for val in dof_dict.itervalues(): dofs.append(pose.torsion(val))
dofs = np.array(dofs)


-9.62295504174


In [38]:
w, v = scipy.linalg.eigh(hessian)
obs = protocols.moves.AddPyMOLObserver(pose, True)
pose.assign(huh)
E_min = scorefxn(huh)
print E_min

NameError: name 'huh' is not defined

In [ ]:
print scorefxn(huh)- 0.5*len(w)*np.log(2*np.pi) + 0.5*np.log(scipy.linalg.det(hessian))

In [ ]:
import scipy.integrate

Z = 1

pose.assign(pose_min)
#E_min = scorefxn(minimum)
kT = 1.0
for kk in range(len(w)):
 
    mode = v[:,kk]
    h = np.arange(-30,31,1)
    #fuck = h[:-1]
    #fuck.reverse()
    #h = h+fuck
    #h = np.array(h)
    h_plot = h*np.pi/180

    bar = []


    for hh in h:

        dofs_new = dofs+hh*mode

        for jj, dof in enumerate(dofs_new):

            pose.set_torsion(dof_dict[jj], dof)

        bar.append((scorefxn(pose)-E_min)/kT)

    
    Z *= np.trapz(np.exp(-np.array(bar)), dx = np.pi/180.)
    np.array(bar) 
    pose.assign(minimum)
    plt.figure()
    #plt.subplot(3, 4, kk+1)
    plt.plot(h, np.exp(-np.array(bar)))
   # plt.plot(h, np.exp(-0.5*w[kk]*h_plot**2/kT))
    plt.xlabel('Displacement along normal coordinate (degrees)')
    plt.ylabel('exp(-(E-E0)/kT)')

In [ ]:
print scorefxn(huh) - np.log(Z)

In [ ]:
print min(E_list)

In [ ]:
print pose.epsilon(1)

In [ ]:
pose = protocols.recces.pose_setup_turner('g','')
print pose.epsilon(1)

In [ ]:
minmover.tolerance(1e-7)
minmover.max_iter(100000)
minmover.apply(pose)


In [ ]:
print dof_lists

In [ ]:
print dof_dict
from scipy.special import gamma
from scipy.special import logsumexp
south_syn_min.assign(pose_min)
print scorefxn(pose_min)

In [ ]:
hessian_ss = calculate_hessian(south_anti_min, scorefxn, dof_dict)
print np.linalg.det(hessian)

In [ ]:
print scorefxn(pose_min) + 0.5*np.log(np.linalg.det(hessian)) - 3.5*np.log(2*np.pi)
E_min = scorefxn(pose_min)
E_min_na = scorefxn(north_anti_min)
E_min_ns = scorefxn(north_syn_min)
E_min_sa = scorefxn(south_anti_min)
E_min_ss = scorefxn(south_syn_min)
print E_min_na, E_min_ns, E_min_sa, E_min_ss


In [ ]:
matt_guess = lambda E: (2.0*np.pi)**(len(w)/2)*(E-E_min)**(len(w)/2 - 1)/gamma(len(w)/2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min, 0.5)#2*(2*np.pi)**((len(w)-1)/2)*(2*(E-E_min))**(len(w)/2-1)/scipy.misc.factorial2(len(w)-2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min,0.5)
matt_guess_na = lambda E: (2.0*np.pi)**(len(w)/2)*(E-E_min_na)**(len(w)/2 - 1)/gamma(len(w)/2)/np.sqrt(np.linalg.det(hessian_na))*np.heaviside(E-E_min_na, 0.5)#2*(2*np.pi)**((len(w)-1)/2)*(2*(E-E_min))**(len(w)/2-1)/scipy.misc.factorial2(len(w)-2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min,0.5)
matt_guess_ns = lambda E: (2.0*np.pi)**(len(w)/2)*(E-E_min_ns)**(len(w)/2 - 1)/gamma(len(w)/2)/np.sqrt(np.linalg.det(hessian_ns))*np.heaviside(E-E_min_ns, 0.5)#2*(2*np.pi)**((len(w)-1)/2)*(2*(E-E_min))**(len(w)/2-1)/scipy.misc.factorial2(len(w)-2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min,0.5)
matt_guess_sa = lambda E: (2.0*np.pi)**(len(w)/2)*(E-E_min_sa)**(len(w)/2 - 1)/gamma(len(w)/2)/np.sqrt(np.linalg.det(hessian_sa))*np.heaviside(E-E_min_sa, 0.5)#2*(2*np.pi)**((len(w)-1)/2)*(2*(E-E_min))**(len(w)/2-1)/scipy.misc.factorial2(len(w)-2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min,0.5)
matt_guess_ss = lambda E: (2.0*np.pi)**(len(w)/2)*(E-E_min_ss)**(len(w)/2 - 1)/gamma(len(w)/2)/np.sqrt(np.linalg.det(hessian_ss))*np.heaviside(E-E_min_ss, 0.5)#2*(2*np.pi)**((len(w)-1)/2)*(2*(E-E_min))**(len(w)/2-1)/scipy.misc.factorial2(len(w)-2)/np.sqrt(np.linalg.det(hessian))*np.heaviside(E-E_min,0.5)

In [ ]:
plt.plot(hist[1][500:1500], matt_guess_na(hist[1][500:1500])*np.exp(-hist[1][500:1500]))
plt.plot(hist[1][500:1500], matt_guess_ns(hist[1][500:1500])*np.exp(-hist[1][500:1500]))
plt.plot(hist[1][500:1500], matt_guess_sa(hist[1][500:1500])*np.exp(-hist[1][500:1500]))
plt.plot(hist[1][500:1500], matt_guess_ss(hist[1][500:1500])*np.exp(-hist[1][500:1500]))
plt.plot(sim._dos_scores[500:1500], np.exp(-sim._dos_scores[500:1500])*sim._dos[500:1500])
plt.legend(['NA','NS','SA','SS','RECCES'])

In [ ]:
print -0.6163*logsumexp(-hist[1][:], b=matt_guess_na(hist[1][:])) -0.6163*logsumexp(-hist[1][:], b=matt_guess_ns(hist[1][:])) -0.6163*logsumexp(-hist[1][:], b=matt_guess_sa(hist[1][:])) -0.6163*logsumexp(-hist[1][:], b=matt_guess_ss(hist[1][:]))


In [ ]:
pose = rosetta.protocols.recces.pose_setup_turner('gg',''

In [ ]:
print pose.torsion(TorsionID(2, BB, 4)), pose.torsion(TorsionID(1, CHI, 2))

In [ ]:
scorefxn(pose)

In [ ]:
print pose

In [ ]:
print pose.alpha(2), pose.beta(2), pose.gamma(2), pose.delta(1), pose.epsilon(1), pose.zeta(1)

In [ ]:
huh = pose_from_file('./../recces_rosetta/results/min.pdb')
print huh.torsion(TorsionID(1, CHI, 2)), huh.torsion(TorsionID(1, CHI, 3))

In [ ]:
print 
pmm.apply(huh)
print huh.epsilon(1), huh.zeta(1), huh.alpha(2), huh.beta(2), huh.gamma(1), huh.delta(2), huh.chi(1), huh.chi(2)

In [ ]:
print huh.alpha(2), huh.beta(2), huh.gamma(2), huh.epsilon(1), huh.zeta(1)

In [ ]:
scorefxn(huh)

In [ ]:
pose.set_ring_conformation('north')

In [ ]:
print pose.

In [ ]:
pose = pose_from_file('./../../das-lab/recces_rosetta/results/ag/ST/ST_637000.pdb')

In [ ]:
print pose.delta(1), pose.torsion(TorsionID(1, CHI, 2)), pose.torsion(TorsionID(1, CHI, 3))
print pose.delta(2), pose.torsion(TorsionID(2, CHI, 2)), pose.torsion(TorsionID(2, CHI, 3))

In [ ]:
# ag/ST/367000, 38000

# C2'-endo
print 145.521118169, -34.8252844774, 155.978251202

# C3'-endo
print 81.1950530464, 35.8703398534, 94.8555221911

In [ ]:
pose = protocols.recces.pose_setup_turner('gg','')

In [ ]:
pmm.apply(pose)
pose.set_torsion(TorsionID(1, BB, 4), 145.5)
pose.set_torsion(TorsionID(1, CHI, 2), -34.8)
pose.set_torsion(TorsionID(1, CHI, 3), 155.98)

In [ ]:
pmm.apply(pose)

In [ ]:
pose.set_chi(1, -56.6)

In [ ]:
pmm.apply(pose)

In [ ]:
print -0.6163*(np.log(90260*2+108.954*2) + 7*np.log(2*np.pi/3) + 2*np.log(2))

In [ ]:
pmm.apply(pose_min)

In [ ]:
print scorefxn(pose_min)

In [ ]:
print E_list

In [ ]:
print min(E_list)


In [ ]:
print E_min

In [ ]:
print rna_hires(pose_min)

In [ ]:
pmm.apply(pose_min)

In [ ]:
print pose_min.epsilon(1), pose_min.zeta(1), pose_min.alpha(2), pose_min.beta(2), pose_min.gamma(2)
print pose_min.delta(1), pose_min.delta(2)

In [ ]:
pose_min.dump_pdb('./../gg_fuck.pdb')

In [ ]:
pose = pose_from_file('./../gg.pdb')
pmm.apply(pose)

In [ ]:
options = scorefxn.energy_method_options()
print options.hbond_options()
options.etable_options().no_lk_polar_desolvation = False
scorefxn.set_energy_method_options(options)
scorefxn.show(pose)

In [ ]:
pmm.apply(pose)


In [ ]:
 print 0.6163*(-np.log(2*90206 + 2*108.954) - 7*np.log(2*np.pi/3) - 2*np.log(2))

In [ ]:
print movemap


In [ ]:
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner')

In [ ]:
print scorefxn

In [ ]:
scorefxn.show(pose_min)

In [ ]:
print pose

In [ ]:
print pose.chi(1)

In [ ]:
N9 = AtomID(1, 19)
C1 = AtomID(1, 8)

In [ ]:
pose.conformation().bond_length(C1, N9)

In [ ]:
print pose

In [ ]:
pyrosetta.rosetta.protocols.constraint_generator

In [ ]:
import pyrosetta.rosetta.protocols.moves.ConstraintSetMover()

In [ ]:
fuck = ConstraintSetMover()

In [ ]:
fuck.set_id(TorsionID(1, CHI, 1))

In [ ]:
print pose.residue(1)

In [ ]:
import pyrosetta.rosetta.protocols.simple_moves

In [ ]:
set_constraint = pyrosetta.rosetta.protocols.simple_moves.ConstraintSetMover()

In [ ]:
set_constraint.constraint_file('./cst_file.cst')
dihedral_constraint_function = core.scoring.func.CircularHarmonicFunc(pose.chi(2), 0.35)
shit = core.scoring.constraints.DihedralConstraint(core.id.AtomID(1,1), core.id.AtomID(1,2), core.id.AtomID(1,3), core.id.AtomID(1,4), dihedral_constraint_function)

In [ ]:
pose = protocols.recces.pose_setup_turner('gg','')
pose.dump_pdb('shit.pdb')
print core.id.AtomID(23,1)
print pose.residue(2).atom(17)

In [ ]:
atom = pose.residue(2).atom_index('C3\'')
print atom

In [ ]:
atom1 = core.id.AtomID(2, 5)

In [ ]:
print pose.conformation().dof_id_from_torsion_id(TorsionID(2, CHI, 1))

In [ ]:
north_anti_min = Pose()
north_anti_min.assign(pose_min)

In [ ]:
north_syn_min = Pose()
north_syn_min.assign(pose_min)

In [ ]:
south_syn_min = Pose()
south_syn_min.assign(pose_min)

In [ ]:
south_anti_min = Pose()
south_anti_min.assign(pose_min)

In [ ]:
pose.set_epsilon(1, -145.65)
pose.set_zeta(1, -62.90)
pose.set_alpha(2, -55.65)
pose.set_beta(2, 168.81)
pose.set_gamma(2, 65.70)
pose.set_chi(1, 67.90)
pose.set_chi(2, 113.39)

pmm.apply(pose)

In [ ]:
pmm.apply(pose_min)

In [ ]:
pose = protocols.recces.pose_setup_turner('ga','')

In [170]:
ST_min = pose_from_file('./../gg_fuck.pdb')
print scorefxn(pose_min)
print scorefxn(ST_min)

core.import_pose.import_pose: File './../gg_fuck.pdb' automatically determined to be of type PDB
-8.87862835747
-9.62295503538


In [225]:
#print ST_min.torsion(TorsionID(2, CHI, 1))
#print ST_min.torsion(TorsionID(2, CHI, 2))
#print ST_min.torsion(TorsionID(2, CHI, 3))
print ST_min.epsilon(1)
print ST_min.zeta(1)
print ST_min.alpha(2)
print ST_min.beta(2)
print ST_min.gamma(2)
print ST_min.chi(1)
print ST_min.chi(2)
minmover.apply(ST_min)
scorefxn(ST_min)

-140.883216456
22.138404465
150.567962919
177.062269576
51.4570578344
77.9525786908
118.885984783


-9.622955054302551

In [221]:
pose.set_epsilon(1, -150.4)
pose.set_zeta(1, 56.5)
pose.set_alpha(2, 156.6)
pose.set_beta(2, 177.3)
pose.set_gamma(2, 53.5)
pose.set_chi(1, 72.4)
pose.set_chi(2, 119.0)

In [237]:
scorefxn(ST_min)

-9.622955054302551

In [239]:
minmover.apply(ST_min)
scorefxn(ST_min)

-9.622955066752619

In [236]:
print pose.epsilon(1)
print pose.zeta(1)
print pose.alpha(2)
print pose.beta(2)
print pose.gamma(2)
print pose.chi(1)
print pose.chi(2)

-151.195497333
38.3005665217
160.463746881
176.53024886
53.500002256
120.199920903
113.615720013


In [406]:
pose_min.dump_pdb('./../../das-lab/ga.pdb')

True

In [722]:
pmm.apply(pose_min)